# Introduction / Business Problem

# Data

### First, some context

Before I describe the data, let me provide some context as to what I will do with the data.

I want to divide the neighborhoods of Toronto into 4 clusters based on how Upscale and Diverse they are. The 4 clusters will be:
(1) Both Upscale & Diverse
(2) Upscale, but not Diverse
(3) Diverse, but not Upscale
(4) Neither Upscale nor Diverse

Of these, I will focus just on the Upscale & Diverse neighborhoods for my final test. In this final test, I will rank the Upscale & Diverse neighborgoods by how many Indian restaurants they already have, and I will suggest that my friend pick one of the neighborhoods with fewer Indian restaurants.

### Next, an outline of the data

I am going to need the following data from Folium, Foursquare, and Wikipedia.

1) A map of Toronto to get started. This will come from Folium.
2) A basic dataset of boroughs and neighborhoods in Toronto, like the one I created for the Week 3 assignment of Segmenting and Clustering neighborhoods in Toronto. This will come from Wikipedia, just like in the Week 3 assignment.
3) Finally, for the K-means clustering, I will need data on the top venues in each neighborhood within a given radius. This will come from Foursquare, just like in the Week 3 assignment.

Overall, my approach will be similar to the analysis in Week 3 with just two major changes
(1) I care only about the top restaurant venues, rather than all the top venues (i.e., I will be ignoring any venue that is not a restaurant).
(2) I need to define some metric that estimates the degree of diversity of restaurants, and the degree of upscale-ness of restaurants, for the K Means clustering.
    For measuring diversity, I will use the cuisine of each restaurant as raw data. I will get the cuisine from the 'Venue Category' field in Foursquare, which classified restaurants into "Afghan restaurant", "African restaurant", "American restaurant", etc. in the Week 3 Lab of segmenting neighborhoods in New York. The higher the variety in cuisine names, the higher that neighborhood's diversity score will be.
    For measuring upscale-ness, I will use the expensive-ness of each restaurant (using the 1-4 "dollar" scale) as raw data. I am looking for a neighborhood that is dominated by 2-dollar and 3-dollar restaurants, because 4-dollar will be too upscale, and 1-dollar will be too cheap.

### Finally, the code